In [ ]:
!pip install pytorch-lightning -q
!python -m spacy download en_core_web_md -q
!pip install -U deep-translator -q
!pip install -U youtube_transcript_api -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
import torch
from torchmetrics import AUROC
import pandas as pd
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import datetime
import glob
import re
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.datasets import make_multilabel_classification
from sklearn.utils import class_weight
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import resample
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt

import spacy

from deep_translator import GoogleTranslator

from youtube_transcript_api import YouTubeTranscriptApi
import json
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

path = '/content/gdrive/MyDrive/b5/'

Mounted at /content/gdrive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df1 = pd.read_csv(f'{path}Big-Five_Backstage.csv')

df2 = pd.read_csv(f'{path}essays/essays.csv', encoding='cp1252', delimiter=',', quotechar='"')
df2 = df2.rename(columns={'TEXT': 'text',
                        'cEXT': 'Extraversion',
                        'cNEU': 'Neuroticism',
                        'cAGR': 'Agreeableness',
                        'cCON': 'Conscientiousness',
                        'cOPN': 'Openness'})
keys =  ['Extraversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
df2[keys] = df2[keys].replace({'y': 1.0, 'n': 0.0})

In [ ]:
texts = pd.concat([df1['text'], df2['text']], ignore_index=True)
labels = pd.concat([df1[keys], df2[keys]], ignore_index=True)

In [ ]:
texts

0       Hist! All is still. They are not yet here. On ...
1       Nay, dearest love, fear not; no mortal eye beh...
2       Ah, dearest Adelbert, with thee I feel no fear...
3       This summer sky, darkened by storm, is a fit e...
4       Heaven shield me! Whither shall I turn? Alone ...
                              ...                        
5721         I'm home. wanted to go to bed but remembe...
5722         Stream of consiousnesssskdj. How do you s...
5723    It is Wednesday, December 8th and a lot has be...
5724    Man this week has been hellish. Anyways, now i...
5725    I have just gotten off the phone with brady. I...
Name: text, Length: 5726, dtype: object

In [ ]:
labels

,Extraversion,Neuroticism,Agreeableness,Conscientiousness,Openness
0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,1.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...
5721,0.0,1.0,0.0,1.0,0.0
5722,1.0,1.0,0.0,0.0,1.0
5723,0.0,0.0,1.0,0.0,0.0
5724,0.0,1.0,0.0,0.0,1.0


In [ ]:
# mrc_df = pd.read_csv(f'{path}mrc2.csv')
# mrc_df_1 = mrc_df[['word', 'nlet', 'nphon', 'nsyl', 'kf_freq', 'kf_ncats', 'kf_nsamp', 'tl_freq', 'brown_freq', 'fam', 'conc', 'imag', 'meanc', 'meanp', 'aoa', 'status']][451:]

# mrc_status = pd.get_dummies(mrc_df_1['status'], prefix='status').drop('status_ ', axis=1)

# mrc_df_prep = pd.concat([mrc_df_1.drop('status', axis=1), mrc_status], axis=1)
# mrc_df_prep.loc[:, mrc_df_prep.columns != 'word'] = mrc_df_prep.loc[:, mrc_df_prep.columns != 'word'].astype(float)
# mrc_df_prep.reset_index(drop=True, inplace=True)

# def remove_non_letters(word):
#     return re.sub(r'[^a-zA-Z]', '', word)

# word_data = {}

# for index, row in mrc_df_prep.iterrows():
#     word = remove_non_letters(str(row['word']).lower().strip())
#     if word == '':
#         continue
#     if word in word_data:
#         word_data[word]['count'] += 1
#         word_data[word]['data'] += np.array([float(row[col]) for col in mrc_df_prep.columns[1:]])
#     else:
#         word_data[word] = {'count': 1, 'data': np.array([float(row[col]) for col in mrc_df_prep.columns[1:]])}

# for word in word_data:
#     count = word_data[word]['count']
#     if count > 1:
#         word_data[word]['data'] /= count
#     del word_data[word]['count']
#     word_data[word] = word_data[word]['data']

# for word, data in word_data.items():
#     word_data[word] = data.tolist()

# import json

# with open(f'{path}mrc2_dict.json', 'w') as f:
#     json.dump(word_data, f)

In [ ]:
with open(f'{path}mrc2_dict_norm.json', 'r') as f:
    word_data = json.load(f)

In [ ]:
# nlp = spacy.load("en_core_web_md",  enable=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])

# splited_texts = []
# MAX_WORDS_PER_SPLIT = 200
# for idx, text in enumerate(texts):
#     print(f'\r{idx}', end='')
#     sentences = re.split(r'(?<=[.!?])\s+', text)

#     current_split = ""
#     current_word_count = 0
#     for sentence in sentences:
#         str_sen =  sentence.strip()
#         if str_sen != "" and any(char.isalnum() for char in sentence):
#             words_in_sentence = len(str_sen.split())

#             if current_word_count + words_in_sentence <= MAX_WORDS_PER_SPLIT:
#                 current_split += str_sen + " "
#                 current_word_count += words_in_sentence
#             elif current_split.strip() != "":
#                 splited_texts.append(current_split.strip())
#                 current_split = str_sen + " "
#                 current_word_count = words_in_sentence

#     if current_split.strip() != "":
#         splited_texts.append(current_split.strip())

# splited_texts = pd.DataFrame(splited_texts)

# splited_texts = splited_texts.drop(20930)

# texts_mrc = []
# for idx, text in enumerate(splited_texts.values):
#   print(f'\r{idx}', end='')
#   s = np.zeros(27)
#   for word in nlp(text[0]):
#     if word.lemma_ in word_data:
#       s += np.array(word_data[word.lemma_])
#   texts_mrc.append(s.tolist())

# torch.tensor(texts_mrc, device=device)
# torch.save(texts_mrc, f"{path}texts_mrc.pt")

In [ ]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_md", enable="tok2vec")

In [ ]:
indices_to_labels = { 0:'e', 1:'n', 2:'a', 3:'c', 4:'o' }
labels_to_indices = { 'e':0, 'n':1, 'a':2, 'c':3, 'o':4 }

TYPE_TO_PROCESS = 'n'
cur_idx = labels_to_indices[TYPE_TO_PROCESS]

In [ ]:
USE_PREDIFINED_DATA = True
if USE_PREDIFINED_DATA:
  labels_tensors = []
  for i in range(1, 7):
    labels_tensors.append(torch.load(f"{path}spacy-vecs/labels-{i}.pt", map_location=device))
  labels_tensor = torch.cat(labels_tensors)

  label_tensor = labels_tensor[:,cur_idx: cur_idx + 1]

  undersampler = RandomUnderSampler(random_state=42)
  indices, under_label_tensor = undersampler.fit_resample(np.array([range(len(label_tensor))]).T, label_tensor.cpu())

  text_vec_arrs = []
  for i in range(1, 7):
    text_vecs = torch.load(f"{path}spacy-vecs/text_vec_arrs-{i}.pt", map_location='cpu')
    filtered_indices = indices[(indices >= 5000*(i-1)) & (indices < 5000*i)] - 5000*(i-1)
    text_vec_arrs.append(text_vecs[filtered_indices].to(device))

  text_vec_arrs = torch.cat(text_vec_arrs)

  texts_mrc = torch.tensor(torch.load(f"{path}texts_mrc.pt", map_location=device), dtype=torch.float, device=device)[indices.T[0]]
  texts_mrc = texts_mrc / 200
  texts_mrc = torch.nan_to_num(texts_mrc, nan=0)

  train_inputs, test_inputs, train_mrc, test_mrc, train_labels, test_labels = train_test_split(text_vec_arrs, texts_mrc, torch.tensor(np.array([under_label_tensor]), dtype=torch.float, device=device).T, test_size=0.2, random_state=42, stratify = under_label_tensor)
  # labels_tensor = torch.load(f"{path}spacy-vecs/labels.pt").to(device)
  # text_vec_arrs = torch.load(f"{path}spacy-vecs/text_vec_arrs.pt").to(device)
else:
  MAX_WORDS_PER_SPLIT = 200

  splited_texts = []
  splited_labels = []
  for idx, text in enumerate(texts):
      print(f'\r{idx}', end='')
      sentences = re.split(r'(?<=[.!?])\s+', text)

      current_split = ""
      current_word_count = 0
      for sentence in sentences:
          str_sen = sentence.strip()
          if str_sen != "" and any(char.isalnum() for char in sentence):
              words_in_sentence = len(str_sen.split())

              if current_word_count + words_in_sentence <= MAX_WORDS_PER_SPLIT:
                  current_split += str_sen + " "
                  current_word_count += words_in_sentence
              elif current_split.strip() != "":
                  splited_texts.append(current_split.strip())
                  splited_labels.append(labels.values[idx])
                  current_split = str_sen + " "
                  current_word_count = words_in_sentence

      if current_split.strip() != "":
          splited_texts.append(current_split.strip())
          splited_labels.append(labels.values[idx])

  splited_texts = pd.DataFrame(splited_texts)

  splited_labels = pd.DataFrame(splited_labels)

  splited_texts = splited_texts.drop(20930)
  splited_labels = splited_labels.drop(20930)

  splited_texts = pd.DataFrame(splited_texts.values[25000:30000])

  splited_labels = pd.DataFrame(splited_labels.values[25000:30000])

  text_vec_arrs = torch.zeros((len(splited_texts), 200, 300), device=device)
  for idx, text in enumerate(splited_texts.values):
      doc = nlp(text[0])

      word_vectors = torch.tensor(np.array([token.vector.get() for token in doc if token.has_vector and token.is_alpha][:200]), device=device)

      text_vec_arrs[idx, :word_vectors.shape[0]] = word_vectors
      print(f'\r{idx}', end='')

  torch.save(text_vec_arrs, f"{path}spacy-vecs/text_vec_arrs-6.pt")

  labels_tensor = torch.tensor(splited_labels.values, dtype=torch.float).to(device)
  torch.save(labels_tensor, f"{path}spacy-vecs/labels-6.pt")

In [ ]:
texts_mrc.shape

torch.Size([25504, 27])

In [ ]:
text_vec_arrs.shape

torch.Size([25504, 200, 300])

In [ ]:
labels_tensor.shape

torch.Size([28377, 5])

In [ ]:
text_vec_arrs_tensor = text_vec_arrs
text_vec_arrs_tensor

tensor([[[  2.5453,   2.2064,  -0.9130,  ...,   4.0204, -11.6140,   2.9858],
         [ -1.0700,   3.6484,  -4.1973,  ...,   0.4962,  -3.9575,  -0.4187],
         [  0.0498,   9.9706,  -9.7731,  ...,  -3.6775, -10.3520,   5.3838],
         ...,
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],

        [[ -3.9874,  -5.6373,   3.7683,  ...,   2.6906,  -3.2566,  -0.4880],
         [ -0.8310,  -3.4769,   2.1100,  ...,   1.8151,  -3.3340,   3.1847],
         [  0.2896,   2.0112,  -0.4945,  ...,   0.7096,   2.0408,  -2.7906],
         ...,
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],

        [[ -4.4363,  -4.8146,  -1.2260,  ...

In [ ]:
def to_cpu(inputs):
  t = inputs.detach().cpu() if inputs.is_cuda else inputs.detach()
  return t.numpy().astype(float)

In [ ]:
train_inputs.shape

torch.Size([20403, 200, 300])

In [ ]:
train_labels.shape

torch.Size([20403, 1])

In [ ]:
# sums = np.sum(to_cpu(train_labels), axis=0)
# class_weights = len(train_labels) / sums - 1
# print(class_weights)
# class_weights = torch.tensor(1/class_weights, dtype=torch.float).to(device)

# sums = np.sum(to_cpu(test_labels), axis=0)
# class_weights = len(test_labels) / sums - 1
# print(class_weights)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_mrc, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_inputs, test_mrc, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomLayer(nn.Module):
    def __init__(self, size):
        super(CustomLayer, self).__init__()
        self.weights = nn.Parameter(torch.randn(size))
        self.bias = nn.Parameter(torch.randn(size))

    def forward(self, x):
        return x * self.weights.t() + self.bias

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class MyLightningModule(pl.LightningModule):
    # def __init__(self):
    #     super().__init__()
    #     self.loss = torch.nn.BCEWithLogitsLoss()
    #     n_grams = range(1, 6)
    #     out_channels = 32
    #     self.conv_layers = nn.ModuleList([
    #         nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(i, 300), padding=(i-1,0))
    #         for i in n_grams
    #     ])

    #     #self.fc1 = nn.Linear(len(n_grams)*out_channels, len(n_grams)*out_channels)
    #     self.fc2 = nn.Linear(len(n_grams)*out_channels + 27, 1)

    # def forward(self, vectors, mrc):
    #     # x = F.avg_pool2d(x, kernel_size=(6, 1), padding=(3, 0), stride=1)
    #     x = vectors.unsqueeze(1)

    #     conv_outputs = []
    #     for conv in self.conv_layers:
    #         conv_output = F.relu(conv(x))
    #         conv_output = F.max_pool2d(conv_output, kernel_size=(conv_output.size(2), 1))
    #         conv_outputs.append(conv_output)

    #     conv_outputs = torch.cat(conv_outputs, dim=1)
    #     #x = conv_outputs.squeeze()
    #     x = conv_outputs.squeeze()
    #     x = x.view(x.size(0), -1)
    #     #x = torch.cat((x, torch.zeros(x.size(0), 256 - x.size(1), device=x.device)), dim=1)

    #     #x = F.relu(self.fc1(x))

    #     x = self.fc2(torch.cat([x, mrc], dim=1))

    #     return x

    # def __init__(self):
    #     super().__init__()
    #     self.loss = torch.nn.BCEWithLogitsLoss()
    #     n_grams = range(1, 6)
    #     padding = 3
    #     kernel = (6, 1)
    #     self.in_features = (math.floor((200 + 2 * padding - kernel[0])/1) + 1)
    #     classes = 5

    #     self.linears = nn.ModuleList([CustomLayer(self.in_features * 300) for _ in range(classes)])

    #     self.fc2 = nn.Linear(classes*300, 1)

    #     #self.fc1 = nn.Linear(len(n_grams)*out_channels, len(n_grams)*out_channels)
    #     #self.fc2 = nn.Linear(len(n_grams)*out_channels*300, 1)
    #     # self.sigmoid = nn.Sigmoid()


    # def forward(self, x):
    #     #x = x.unsqueeze(1)
    #     x = F.avg_pool2d(x, kernel_size=(6, 1), padding=(3, 0), stride=1)
    #     x = x.view(x.size(0), -1)
    #     linear_results = [F.relu(l(x)) for l in self.linears]
    #     x = torch.stack(linear_results, dim=1).view(x.size(0), len(self.linears), self.in_features, 300)
    #     x = F.max_pool2d(x, kernel_size=(self.in_features, 1))
    #     x = x.view(x.size(0), -1)

    #     x = self.fc2(x)

    #     return x


    def __init__(self):
        super().__init__()
        self.loss = torch.nn.BCEWithLogitsLoss()
        self.padding = (3, 0)
        self.stride = (6, 1)
        self.kernel = (6, 1)
        #self.in_features = (math.floor((200 + 2 * self.padding[0] - self.kernel[0])/self.stride[0]) + 1)
        self.GRU = nn.GRU(300, 300, batch_first=True)
        self.GRU1 = nn.GRU(300, 300, batch_first=True)
        self.fc1 = nn.Linear(200 * 300, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128 + 27, 64)
        self.fc4 = nn.Linear(64, 1)


    def forward(self, vectors, mrc):
        x = vectors.unsqueeze(1)
        #x = F.avg_pool2d(x, kernel_size=self.kernel, padding=self.padding, stride=self.stride)
        x = self.GRU(x.squeeze(1))[0]
        x = self.GRU1(x)[0]
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(torch.cat([x, mrc], dim=1)))
        # x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x


    def training_step(self, batch, batch_idx):
        inputs, mrc, labels = batch
        outputs = self(inputs, mrc)

        loss = self.loss(outputs, labels.float())

        self.log('train_loss', loss, on_step=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, mrc, labels = batch
        outputs = self(inputs, mrc)

        loss = self.loss(outputs, labels.float())

        self.log('val_loss', loss, on_step=False, on_epoch=True)

        # self.val_labels.extend(to_cpu(labels))
        # self.val_outputs.extend((to_cpu(F.sigmoid(outputs))> 0.5).astype(float))


    # def on_validation_epoch_start(self) -> None:
    #     super().on_validation_epoch_start()
    #     self.val_labels = []
    #     self.val_outputs = []
    #     return

    # def on_validation_epoch_end(self):
    #     loss = f1_score(self.val_labels, self.val_outputs, average='macro')
    #     self.log('val_loss', -loss, on_epoch=True)


    def test_step(self, batch, batch_idx):
        inputs, mrc, labels = batch
        outputs = F.sigmoid(self(inputs, mrc))

        mse = torch.mean((outputs - labels)**2)
        self.log('test_mse', mse, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-6)#, weight_decay=1e-6)

    # def configure_optimizers(self):
    #     return torch.optim.SGD(self.parameters(), lr=1e-5)

    # def configure_optimizers(self):
    #     return torch.optim.RMSprop(self.parameters(), lr=1e-5, weight_decay=1e-5)

    # def configure_optimizers(self):
    #     return torch.optim.Adagrad(self.parameters(), lr=1e-5)


In [ ]:
def get_best_checkpoint_path(checkpoint_dir):
    checkpoint_files = glob.glob(f"{checkpoint_dir}/*.ckpt")

    if not checkpoint_files:
        return None

    best_checkpoint_path = None
    best_loss = float('inf')

    for checkpoint_file in checkpoint_files:
        file_parts = checkpoint_file.split('-')
        val_loss = float(file_parts[-1][:-5].split('=')[1])

        if val_loss < best_loss:
            best_loss = val_loss
            best_checkpoint_path = checkpoint_file

    return best_checkpoint_path

In [ ]:
lModel = MyLightningModule()
cur_path = f"{path}RNN/{TYPE_TO_PROCESS}/"
num_epochs = 20
trainer = pl.Trainer(
    # val_check_interval=50,
    check_val_every_n_epoch=1,
    max_epochs=num_epochs,
    log_every_n_steps=50,
    logger=pl.loggers.TensorBoardLogger(f'{cur_path}logs/', name=f'RNN-v1.0'),
    callbacks=[pl.callbacks.ModelCheckpoint(dirpath=f'{cur_path}checkpoints/', filename=f'RNN-v1.0-{{epoch}}-{{val_loss:.3f}}', monitor='val_loss', mode='min', save_top_k=1, verbose=True, enable_version_counter=True)]
)
USE_CHECKPOINT = False
if USE_CHECKPOINT:
  #best_checkpoint_path = get_best_checkpoint_path(f'{cur_path}checkpoints')
  lModel = MyLightningModule.load_from_checkpoint(checkpoint_path=cur_path+"checkpoints/RNN-v1.0-epoch=19-val_loss=0.332.ckpt")
  lModel.configure_optimizers()

trainer.fit(lModel, train_dataloaders=train_loader, val_dataloaders=test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/gdrive/.shortcut-targets-by-id/1-FlGYYbCrLYtFwF16K3n-LPIeS1wfsMY/b5/RNN/n/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type              | Params
-------------------------------------------
0 | loss | BCEWithLogitsLoss | 0     
1 | GRU  | GRU               | 541 K 
2 | GRU1 | GRU               | 541 K 
3 | fc1  | Linear            | 15.4 M
4 | fc2  | Linear            | 32.9 K
5 | fc3  | Linear            | 10.0 K
6

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 2551: 'val_loss' reached 0.48659 (best 0.48659), saving model to '/content/gdrive/.shortcut-targets-by-id/1-FlGYYbCrLYtFwF16K3n-LPIeS1wfsMY/b5/RNN/n/checkpoints/RNN-v1.0-epoch=0-val_loss=0.487.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 17, global step 45918: 'val_loss' reached 0.32434 (best 0.32434), saving model to '/content/gdrive/.shortcut-targets-by-id/1-FlGYYbCrLYtFwF16K3n-LPIeS1wfsMY/b5/RNN/n/checkpoints/RNN-v1.0-epoch=17-val_loss=0.324.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 18, global step 48469: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 51020: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
trainer.test(dataloaders=test_loader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/gdrive/.shortcut-targets-by-id/1-FlGYYbCrLYtFwF16K3n-LPIeS1wfsMY/b5/RNN/n/checkpoints/RNN-v1.0-epoch=17-val_loss=0.324.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at /content/gdrive/.shortcut-targets-by-id/1-FlGYYbCrLYtFwF16K3n-LPIeS1wfsMY/b5/RNN/n/checkpoints/RNN-v1.0-epoch=17-val_loss=0.324.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_mse          │    0.10513895004987717    │
└───────────────────────────┴───────────────────────────┘

[{'test_mse': 0.10513895004987717}]

In [ ]:
label_names = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness'][cur_idx:cur_idx+1]

In [ ]:
def split_text_into_chunks(text, chunk_size=200):
    words = text.split()
    num_chunks = len(words) // chunk_size + (1 if len(words) % chunk_size != 0 else 0)
    chunks = []
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(words))
        chunk = ' '.join(words[start_idx:end_idx])
        chunks.append(chunk)
    return chunks

video_id = "rKWeOfcTN44"
transcript = YouTubeTranscriptApi.get_transcript(video_id)

yt_texts = split_text_into_chunks(" ".join([item['text'] for item in transcript]))

In [ ]:
text = "Hi. What is your name? How old are you?"
text = "Привет. Как тебя зовут? Сколько тебе лет?"
text = GoogleTranslator(source='auto', target='en').translate(text)
print(text)
lModel.eval()
lModel.to(device)
with torch.no_grad():
    input = torch.zeros((200, 300), device=device)
    words = torch.tensor(np.array([token.vector.get() for token in nlp(text) if token.has_vector and token.is_alpha][:200]), device=device)
    input[:words.shape[0]] = words
    outputs = F.sigmoid(lModel(input.unsqueeze(0).unsqueeze(0)))
    print(outputs)
    raw_predicted_labels = outputs.float().tolist()
    predicted_labels = (outputs > 0.5).float().tolist()
    result1 = {label_names[i]: predicted_labels[i] for i in range(len(label_names))}
    result2 = {label_names[i]: raw_predicted_labels[i] for i in range(len(label_names))}
    print(result1)
    print(result2)

Hello. What is your name? How old are you?


TypeError: MyLightningModule.forward() missing 1 required positional argument: 'mrc'

In [ ]:
lModel.eval()
lModel.to(device)
ln = 0
result_values = []
res_lab_values = []
with torch.no_grad():
    for batch in test_loader:
        inputs, mrc, labels = batch
        outputs = F.sigmoid(lModel(inputs, mrc))
        ln += len(outputs)
        for idx in range(len(outputs)):
          result = {label_names[i]: (to_cpu(outputs[idx])> 0.5)[i].astype(float) for i in range(len(label_names))}
          res_lab = {label_names[i]: (to_cpu(labels[idx])> 0.5)[i].astype(float) for i in range(len(label_names))}
          result_values.append(np.array(list(result.values())))
          res_lab_values.append(np.array(list(res_lab.values())))
          # print(res_lab)
          # print(result)
          # print()
pos = np.zeros(len(label_names))
for arr1, arr2 in zip(result_values, res_lab_values):
  pos += (arr1 == arr2).astype(float)
# print(pos/len(test_loader.dataset))
# print(list(map(lambda x: x, result_values)))
print([v[0] for v in result_values])
# f1_scores = f1_score(res_lab_values, result_values, average=None)

# print("F1-меры для каждого класса:", f1_scores)

# average_f1 = f1_score(res_lab_values, result_values, average='macro')

# print("Среднее значение F1-меры:", average_f1)

f1 = f1_score(res_lab_values, result_values)

print("Значение F1-меры:", f1)
accuracy_score(res_lab_values, result_values)


[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0,

0.847676926092923